In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Flatten 
from tensorflow.keras.layers import Conv2D 
from tensorflow.keras.layers import MaxPooling2D 
from tensorflow.keras.applications import VGG16
import matplotlib.pyplot as plt

In [ ]:
# get generators from py file 
%run image_generators.py

In [ ]:

model = Sequential()
# define 3x3 filter window sizes. Create 32 filters.
model.add(Conv2D(filters=32,
                        kernel_size=(3, 3),
                        activation='relu',
                        input_shape=(224, 224, 3)))


# max pool in 2x2 window
model.add(MaxPooling2D(pool_size=(2, 2)))
# define 3x3 filter window sizes. Create 64 filters.
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))

# connect nodes all nodes with dense layers 
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='sigmoid'))


bce = tf.keras.losses.BinaryCrossentropy()


model.compile(optimizer='adam', loss=bce,  metrics=['accuracy'])

In [ ]:
# fit model - CT

CT_base_model = model.fit(
        ct_train_generator,
        steps_per_epoch=100,verbose = 1,validation_data = ct_validation_generator, 
        epochs=10)

In [ ]:
# fit model - X-Ray
XR_base_model = model.fit(
        XR_train_generator,
        steps_per_epoch=100,verbose =1,validation_data = xr_validation_generator, 
        epochs=10)

In [ ]:
# base model evaluation - CT scans

y_pred = model.predict(ct_train_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(ct_train_generator, axis=1)

cm = confusion_matrix(y_true_classes, y_pred_classes)
print(cm)



y_pred = model.predict(ct_train_generator)
fpr, tpr, thresholds = roc_curve(ct_train_generator, y_pred)
roc_auc = auc(fpr, tpr)

import matplotlib.pyplot as plt

plt.plot(fpr, tpr)
         
         
         

In [ ]:
plt.plot(fpr, tpr)

In [ ]:
## vgg16

vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

vggmodel = tf.keras.models.Sequential()
vggmodel.add(vgg_model)
vggmodel.add(tf.keras.layers.Flatten())
vggmodel.add(tf.keras.layers.Dense(256, activation='relu'))
vggmodel.add(tf.keras.layers.Dropout(0.5))
vggmodel.add(tf.keras.layers.Dense(1, activation='softmax'))

vggmodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
vggmodel.fit_generator(ct_train_generator,
                    steps_per_epoch=20,
                    epochs=10,
                    validation_data=ct_validation_generator)

In [ ]:
vggmodel.fit_generator(xr_train_generator,
                    steps_per_epoch=20,
                    epochs=10,
                    validation_data=xr_validation_generator)

In [ ]:
# tuning base model 

model2 = Sequential()

model2.add(Conv2D(filters=32,
                        kernel_size=(3, 3),
                        activation='relu',
                        input_shape=(224, 224, 3)))



model2.add(MaxPooling2D(pool_size=(2, 2)))

model2.add(Conv2D(64, (3, 3), activation='relu'))
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(64, (3, 3), activation='relu'))


model2.add(Flatten())
model2.add(Dense(64, activation='relu'))
model2.add(Dense(8, activation='tahn'))
model2.add(Dense(2, activation='sigmoid'))
